In [1]:
# Dependencies
import pandas as pd
import requests
import json
import os
import csv
import prettyprint as pp
# from apiclient import errors

# Google developer API key
from api_keys import gkey

In [9]:
world_happiness = "world happiness data.csv"
location_types = "Location_Types.csv"
world_happiness_df = pd.read_csv(world_happiness, encoding="UTF-8")
location_types_df = pd.read_csv(location_types, encoding="UTF-8")
 # base url
places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [5]:
health_location_types = location_types_df.loc[location_types_df["sub_type"]
                                               =="Health",:]
health_types_list = []
health_location_types.reset_index(inplace=True)
for i in range(len(health_location_types)):
    health_types_list.append(health_location_types.loc[i,"location_types"])

health_types_list

['dentist', 'doctor', 'gym', 'hospital', 'pharmacy', 'physiotherapist']

In [6]:
base_url = "https://maps.googleapis.com/maps/api/geocode/json"
coord=[]
cities=["Helsinki","Oslo","Ottawa","Washington D.C","Rome","Tokyo","City of Victoria","Beijing","Dhaka"",New Delhi","Gaborone","Kabul"]
for city in cities:
    params = {"address": city, "key":gkey}
    response = requests.get(base_url, params=params)
    places_data = response.json()   
    lat=places_data["results"][0]["geometry"]["location"]["lat"]
    long=places_data["results"][0]["geometry"]["location"]["lng"]
    coord.append(f"{lat},{long}")
coord

['60.16985569999999,24.9383791',
 '59.9138688,10.7522454',
 '45.4215296,-75.69719309999999',
 '38.9071923,-77.0368707',
 '41.9027835,12.4963655',
 '35.6761919,139.6503106',
 '44.8608086,-93.660106',
 '39.90419989999999,116.4073963',
 '28.6461322,77.1964488',
 '-24.6282079,25.9231471',
 '34.5553494,69.207486']

In [10]:
#Use for loop to calculate hospitals

target_type ="hospital"
count_hospitals=[]

for lat in coord:
    params = {"location":lat,"key": gkey,"keyword":target_type, "radius":750}
    places_data = requests.get(places_url, params=params).json()
    count_hospitals.append(len(places_data["results"]))
    
count_hospitals

[4, 1, 1, 2, 1, 0, 0, 3, 9, 1, 1]

In [11]:
#Use for loop to calculate dentists

target_type ="dentist"
count_dentists=[]

for lat in coord:
    params = {"location":lat,"key": gkey,"keyword":target_type, "radius":750}
    places_data = requests.get(places_url, params=params).json()
    count_dentists.append(len(places_data["results"]))

count_dentists

[20, 20, 20, 20, 20, 9, 3, 1, 8, 1, 3]

In [ ]:
health_types_list

In [12]:
#Use for loop to calculate doctors
target_type ="doctor"
count_doctors=[]

for lat in coord:
    params = {"location":lat,"key": gkey,"keyword":target_type, "radius":750}
    places_data = requests.get(places_url, params=params).json()
    count_doctors.append(len(places_data["results"]))

count_doctors

[20, 20, 20, 20, 20, 13, 3, 9, 20, 6, 2]

In [16]:
#Use for loop to calculate doctors
target_type ="gym"
count_gyms=[]

for lat in coord:
    params = {"location":lat,"key": gkey,"keyword":target_type, "radius":750}
    places_data = requests.get(places_url, params=params).json()
    count_gyms.append(len(places_data["results"]))

count_gyms

[20, 20, 20, 20, 13, 10, 2, 11, 20, 2, 2]

In [13]:

#Use for loop to calculate doctors
target_type ="pharmacy"
count_pharmacies=[]

for lat in coord:
    params = {"location":lat,"key": gkey,"keyword":target_type, "radius":750}
    places_data = requests.get(places_url, params=params).json()
    count_pharmacies.append(len(places_data["results"]))

count_pharmacies

[20, 20, 18, 20, 20, 20, 0, 14, 20, 6, 3]

In [14]:

#Use for loop to calculate doctors
target_type ="physiotherapist"
count_physio=[]

for lat in coord:
    params = {"location":lat,"key": gkey,"keyword":target_type, "radius":750}
    places_data = requests.get(places_url, params=params).json()
    count_physio.append(len(places_data["results"]))

    
count_physio

[9, 7, 13, 18, 16, 0, 5, 0, 1, 2, 0]

In [17]:

health_dict = {"Cities":cities,"Hospitals":count_hospitals,"Doctors":count_doctors,"Dentists":count_dentists,
              "Gyms":count_gyms,"Pharmacies":count_pharmacies,"Physiotherapists":count_physio}

health_data_df = pd.DataFrame(health_dict)
health_data_df

,Cities,Hospitals,Doctors,Dentists,Gyms,Pharmacies,Physiotherapists
0,Helsinki,4,20,20,20,20,9
1,Oslo,1,20,20,20,20,7
2,Ottawa,1,20,20,20,18,13
3,Washington D.C,2,20,20,20,20,18
4,Rome,1,20,20,13,20,16
5,Tokyo,0,13,9,10,20,0
6,City of Victoria,0,3,3,2,0,5
7,Beijing,3,9,1,11,14,0
8,"Dhaka,New Delhi",9,20,8,20,20,1
9,Gaborone,1,6,1,2,6,2


In [19]:
#sum the totals of each row
sum_rows= health_data_df.sum(axis=1)
sum_rows

0      93
1      88
2      92
3     100
4      90
5      52
6      13
7      38
8      78
9      18
10     11
dtype: int64

In [20]:
#Add to dataframe
health_data_df["Total"]=sum_rows
health_data_df

,Cities,Hospitals,Doctors,Dentists,Gyms,Pharmacies,Physiotherapists,Total
0,Helsinki,4,20,20,20,20,9,93
1,Oslo,1,20,20,20,20,7,88
2,Ottawa,1,20,20,20,18,13,92
3,Washington D.C,2,20,20,20,20,18,100
4,Rome,1,20,20,13,20,16,90
5,Tokyo,0,13,9,10,20,0,52
6,City of Victoria,0,3,3,2,0,5,13
7,Beijing,3,9,1,11,14,0,38
8,"Dhaka,New Delhi",9,20,8,20,20,1,78
9,Gaborone,1,6,1,2,6,2,18
